In [1]:
import numpy as np
from sklearn.metrics import accuracy_score as accuracy
from sklearn.metrics import precision_score as precision
from sklearn.metrics import f1_score as f1
from sklearn.metrics import recall_score as recall
from sklearn.metrics import mean_absolute_percentage_error as mape

In [2]:
import pandas as pd
from tqdm.notebook import tqdm
import itertools
import random
tqdm.pandas()

## Подготовка данных

In [3]:
data = pd.read_csv("data_preprocessed.csv")
data

,Unnamed: 0,pharmacy,card_num,receipt_date,product,cashier_num,receipt_num,application,retail_price,receipt_unique,month
0,3,0,2363656,2018-01-05,784213,0,1,379517,530.74,2018-01-05 1 0,1
1,4,0,2363656,2018-01-05,784213,0,1,379517,530.74,2018-01-05 1 0,1
2,7,0,2364030,2018-01-04,768150,0,3,379753,79.31,2018-01-04 3 0,1
3,8,0,2364030,2018-01-04,778137,0,3,379753,217.95,2018-01-04 3 0,1
4,9,0,2364030,2018-01-15,772668,0,4,379956,745.22,2018-01-15 4 0,1
...,...,...,...,...,...,...,...,...,...,...,...
3330950,3937373,75,10392076,2018-06-06,768485,208,38,379411,929.79,2018-06-06 38 75,6
3330951,3937374,75,10541072,2018-06-07,778885,208,38,379411,828.36,2018-06-07 38 75,6
3330952,3937375,75,10545842,2018-06-28,763319,208,6,379741,5071.59,2018-06-28 6 75,6
3330953,3937376,75,10882494,2018-06-24,781936,208,6,379699,1764.91,2018-06-24 6 75,6


In [4]:
test = pd.read_csv('СА_test.csv',sep=',')
test = test.rename(columns={'Применение':'application', 'Розничная цена':'retail_price','№ карты':'card_num','Аптека':'pharmacy','Дата чека':'receipt_date','Товар':'product','№ кассы':'cashier_num','№ чека':'receipt_num','Применение':'application'})
test['Unnamed: 0'] = 1
test['month'] = 9
test

,pharmacy,card_num,receipt_date,product,Код товара,cashier_num,receipt_num,application,retail_price,Unnamed: 0,month
0,0,2159440,2018-09-17,776743,3762,0,18,379931,506.80,1,9
1,0,2159440,2018-09-17,778684,756,0,18,379930,609.84,1,9
2,0,2159440,2018-09-21,773408,79992,0,6,379562,2366.74,1,9
3,0,2224918,2018-09-19,781468,9609,0,6,379976,1606.00,1,9
4,0,2364030,2018-09-20,769090,82659,0,18,379979,177.08,1,9
...,...,...,...,...,...,...,...,...,...,...,...
557391,75,8972316,2018-09-12,763306,90435,208,6,379737,10335.37,1,9
557392,75,8988894,2018-09-01,764528,39102,208,6,379885,1890.07,1,9
557393,75,8988894,2018-09-01,764536,101376,208,6,379885,1191.00,1,9
557394,75,8988894,2018-09-01,782021,15027,208,6,379885,966.61,1,9


In [5]:
data = pd.concat([data,test])
data

,Unnamed: 0,pharmacy,card_num,receipt_date,product,cashier_num,receipt_num,application,retail_price,receipt_unique,month,Код товара
0,3,0,2363656,2018-01-05,784213,0,1,379517,530.74,2018-01-05 1 0,1,NaN
1,4,0,2363656,2018-01-05,784213,0,1,379517,530.74,2018-01-05 1 0,1,NaN
2,7,0,2364030,2018-01-04,768150,0,3,379753,79.31,2018-01-04 3 0,1,NaN
3,8,0,2364030,2018-01-04,778137,0,3,379753,217.95,2018-01-04 3 0,1,NaN
4,9,0,2364030,2018-01-15,772668,0,4,379956,745.22,2018-01-15 4 0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
557391,1,75,8972316,2018-09-12,763306,208,6,379737,10335.37,NaN,9,90435.0
557392,1,75,8988894,2018-09-01,764528,208,6,379885,1890.07,NaN,9,39102.0
557393,1,75,8988894,2018-09-01,764536,208,6,379885,1191.00,NaN,9,101376.0
557394,1,75,8988894,2018-09-01,782021,208,6,379885,966.61,NaN,9,15027.0


## Фича иженирнг

In [6]:
dfs = {}
for i in range(1, 10):
    dfs[f'data_{i}'] = (
        data[data['month']<i][["card_num", "application",'Unnamed: 0']]
    .groupby(["card_num", "application"])
    .count()
    .reset_index()
    )
    dfs[f'data_{i}']['month'] = i
    dfs[f'data_{i}'] = dfs[f'data_{i}'].sort_values(by=['card_num','application','month'], ascending=True)
    dfs[f'data_{i}']['buys'] = dfs[f'data_{i}']['Unnamed: 0']
    dfs[f'data_{i}']['buys_app_this_month'] = dfs[f'data_{i}']['buys'] - dfs[f'data_{i}']['buys'].shift(1,fill_value=0)
    dfs[f'data_{i}']['buys_app_this_month'][dfs[f'data_{i}']['buys_app_this_month'] <= 0] = 0
    dfs[f'data_{i}']['target'] = 0
    dfs[f'data_{i}']['target'][dfs[f'data_{i}']['buys_app_this_month'] > 0] = 1

    dfs[f'data_{i}'].drop('Unnamed: 0',axis=1,inplace=True)

In [7]:
full = pd.DataFrame(columns = dfs['data_1'].columns)
for df in dfs:
    full = pd.concat([full, dfs[df]])

In [8]:
full = full.sort_values(by=['card_num','application','month'], ascending=True)
full

,card_num,application,month,buys,buys_app_this_month,target
0,2,379411,2,1,1,1
0,2,379411,3,1,1,1
0,2,379411,4,4,4,1
0,2,379411,5,4,4,1
0,2,379411,6,4,4,1
...,...,...,...,...,...,...
1172980,403429824,379939,5,1,0,0
1391098,403429824,379939,6,1,0,0
1583327,403429824,379939,7,1,0,0
1756411,403429824,379939,8,1,0,0


In [9]:
dfs_1 = {}
for i in range(1, 10):
    dfs_1[f'data_{i}'] = (
        data[data['month']<i][["card_num",'Unnamed: 0']]
    .groupby(["card_num"])
    .count()
    .reset_index()
    )
    dfs_1[f'data_{i}']['month'] = i
    dfs_1[f'data_{i}']['buys_all'] = dfs_1[f'data_{i}']['Unnamed: 0']
    dfs_1[f'data_{i}'].drop('Unnamed: 0',axis=1,inplace=True)


In [10]:
full_1 = pd.DataFrame(columns = dfs_1['data_1'].columns)
for df in dfs_1:
    full_1 = pd.concat([full_1, dfs_1[df]])
full_1

,card_num,month,buys_all
0,2,2,26
1,4,2,92
2,6,2,71
3,8,2,4
4,10,2,17
...,...,...,...
122863,403410880,9,9
122864,403410984,9,6
122865,403410988,9,32
122866,403410996,9,49


In [11]:
dfs_2 = {}
for i in range(1, 10):
    dfs_2[f'data_{i}'] = (
        data[data['month']<i][["card_num", "application",'Unnamed: 0']]
    .groupby(["card_num", "application"])
    .count()
    .reset_index()
    )
    dfs_2[f'data_{i}']['month'] = i
    dfs_2[f'data_{i}']['buys_application_all'] = dfs_2[f'data_{i}']['Unnamed: 0']
    dfs_2[f'data_{i}'].drop('Unnamed: 0',axis=1,inplace=True)


In [12]:
full_2 = pd.DataFrame(columns = dfs_2['data_1'].columns)
for df in dfs_2:
    full_2 = pd.concat([full_2, dfs_2[df]])
full_2

,card_num,application,month,buys_application_all
0,2,379411,2,1
1,2,379440,2,1
2,2,379497,2,1
3,2,379525,2,5
4,2,379646,2,1
...,...,...,...,...
1922953,403429824,379929,9,1
1922954,403429824,379931,9,1
1922955,403429824,379933,9,1
1922956,403429824,379937,9,2


In [13]:
dfs_3 = {}
for i in range(1, 10):
    dfs_3[f'data_{i}'] = (
        data[data['month']<i][['month',"card_num",'Unnamed: 0']]
    .groupby(["card_num", "month"])
    .count()
    .reset_index()
    )
    dfs_3[f'data_{i}'] = dfs_3[f'data_{i}'][dfs_3[f'data_{i}']['month']==i-1]
    dfs_3[f'data_{i}']['month'] = i
    dfs_3[f'data_{i}']['buys_month_prev'] = dfs_3[f'data_{i}']['Unnamed: 0']
    dfs_3[f'data_{i}'].drop('Unnamed: 0',axis=1,inplace=True)


In [14]:
full_3 = pd.DataFrame(columns = dfs_3['data_1'].columns)
for df in dfs_3:
    full_3 = pd.concat([full_3, dfs_3[df]])
full_3

,card_num,month,buys_month_prev
0,2,2,26
1,4,2,92
2,6,2,71
3,8,2,4
4,10,2,17
...,...,...,...
552105,403410838,9,7
552111,403410868,9,13
552117,403410874,9,3
552122,403410984,9,3


In [15]:
data_all = full.merge(full_1,how='left',on=['card_num','month'])
data_all = data_all.merge(full_2,how='left',on=['card_num','month','application'])
data_all = data_all.merge(full_3,how='left',on=['card_num','month'])


In [16]:
data_all = data_all.fillna(0)

In [17]:
data_all['period'] = data_all['buys_application_all']/8
data_all['percent_all'] = data_all['buys_application_all'] / data_all['buys_all']
data_all

,card_num,application,month,buys,buys_app_this_month,target,buys_all,buys_application_all,buys_month_prev,period,percent_all
0,2,379411,2,1,1,1,26,1,26,0.125,0.038462
1,2,379411,3,1,1,1,38,1,12,0.125,0.026316
2,2,379411,4,4,4,1,82,4,44,0.500,0.048780
3,2,379411,5,4,4,1,101,4,19,0.500,0.039604
4,2,379411,6,4,4,1,151,4,50,0.500,0.026490
...,...,...,...,...,...,...,...,...,...,...,...
9678968,403429824,379939,5,1,0,0,12,1,0,0.125,0.083333
9678969,403429824,379939,6,1,0,0,13,1,1,0.125,0.076923
9678970,403429824,379939,7,1,0,0,13,1,0,0.125,0.076923
9678971,403429824,379939,8,1,0,0,13,1,0,0.125,0.076923


## Clusterisation results

In [19]:
cluster = pd.read_csv('dbscan_clusters.csv')
cluster.head()

,card_num,cluster_num
0,2,-1
1,4,-1
2,6,-1
3,8,-1
4,10,-1


In [20]:
data_all2 = data_all.merge(cluster, how='inner', on=['card_num'])
data_all2.head()

,card_num,application,month,buys,buys_app_this_month,target,buys_all,buys_application_all,buys_month_prev,period,percent_all,cluster_num
0,2,379411,2,1,1,1,26,1,26,0.125,0.038462,-1
1,2,379411,3,1,1,1,38,1,12,0.125,0.026316,-1
2,2,379411,4,4,4,1,82,4,44,0.500,0.048780,-1
3,2,379411,5,4,4,1,101,4,19,0.500,0.039604,-1
4,2,379411,6,4,4,1,151,4,50,0.500,0.026490,-1


# Разделение на трейн/тест по месяцу

In [21]:
X_train = data_all[data_all['month']<9].drop(['target','buys_app_this_month'],axis=1).reset_index().drop('index',axis=1)
X_test = data_all[data_all['month']<9].drop(['card_num','application','month','buys','buys_all','buys_application_all','buys_month_prev','buys_month_prev','period','percent_all','buys_app_this_month'],axis=1).reset_index().drop('index',axis=1)
y_train = data_all[(data_all['month']==9)].drop(['target','buys_app_this_month'],axis=1).reset_index().drop('index',axis=1)
y_test = data_all[(data_all['month']==9)].drop(['card_num','application','month','buys','buys_all','buys_application_all','buys_month_prev','buys_month_prev','period','percent_all','buys_app_this_month'],axis=1).reset_index().drop('index',axis=1)

In [40]:
X_train2 = data_all2[data_all2['month']<9].drop(['target','buys_app_this_month'],axis=1).reset_index().drop('index',axis=1)
X_test2 = data_all2[data_all2['month']<9].drop(['card_num','application','month','buys','buys_all','buys_application_all','buys_month_prev','buys_month_prev','period','percent_all','buys_app_this_month', 'cluster_num'],axis=1).reset_index().drop('index',axis=1)
y_train2 = data_all2[(data_all2['month']==9)].drop(['target','buys_app_this_month'],axis=1).reset_index().drop('index',axis=1)
y_test2 = data_all2[(data_all2['month']==9)].drop(['card_num','application','month','buys','buys_all','buys_application_all','buys_month_prev','buys_month_prev','period','percent_all','buys_app_this_month', 'cluster_num'],axis=1).reset_index().drop('index',axis=1)

In [23]:
X_train = X_train.values
X_train = X_train[:,:]
X_test = X_test.values
X_test = X_test[:,:]
y_train = y_train.values
y_train = y_train[:,:]

In [34]:
X_train2 = X_train2.values
X_train2 = X_train2[:,:]
X_test2 = X_test2.values
X_test2 = X_test2[:,:]
y_train2 = y_train2.values
y_train2 = y_train2[:,:]

## Обучение

In [25]:
import xgboost

In [26]:
def wape(actual, forecast):
    se_abs = abs(actual.to_numpy().reshape(1, -1)[0] - forecast)

    sum_actual = actual.sum().to_numpy()[0]

    return se_abs.sum() / sum_actual

In [27]:
xgb_r = xgboost.XGBRegressor(objective ='binary:logistic',max_depth=15, learning_rate=0.01,
                  n_estimators = 400, seed = 123, alpha=0, eval_metric=wape)
 
# Fitting the model
xgb_r.fit(X_train, X_test)

XGBRegressor(alpha=0, base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=<function wape at 0x7ff921152af0>, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.01, max_bin=256,
             max_cat_to_onehot=4, max_delta_step=0, max_depth=15, max_leaves=0,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=400, n_jobs=0, num_parallel_tree=1,
             objective='binary:logistic', predictor='auto', random_state=123, ...)

In [36]:
xgb_r2 = xgboost.XGBRegressor(objective ='binary:logistic',max_depth=15, learning_rate=0.01,
                  n_estimators = 400, seed = 123, alpha=0, eval_metric=wape)
 
# Fitting the model
xgb_r2.fit(X_train2, X_test2)

XGBRegressor(alpha=0, base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=<function wape at 0x7ff921152af0>, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.01, max_bin=256,
             max_cat_to_onehot=4, max_delta_step=0, max_depth=15, max_leaves=0,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=400, n_jobs=0, num_parallel_tree=1,
             objective='binary:logistic', predictor='auto', random_state=123, ...)

In [37]:
pred = xgb_r.predict(y_train)
pred2 = xgb_r2.predict(y_train2)

In [42]:
print('with clusters: ',accuracy(y_test2, np.where(pred >= 0.53, 1, 0)))
print('without clusters: ',accuracy(y_test, np.where(pred >= 0.53, 1, 0)))

with clusters:  0.9304373782474709
without clusters:  0.9304373782474709


In [49]:
wape(y_test2, pred2)
print('with clusters WAPE: ', wape(y_test2,np.where(pred2 >= 0.53, 1, 0)))

with clusters WAPE:  0.29463132797729485


In [44]:
wape(y_test, pred)
print('without clusters WAPE: ', wape(y_test, np.where(pred >= 0.53, 1, 0)))

without clusters WAPE:  0.29475692898915423


In [27]:
wape(y_test,np.where(pred >= 0.53, 1, 0))

0.29475692898915423

In [28]:
for_csv = data_all[(data_all['month']==9)]

In [29]:
for_csv = for_csv[['card_num','application','target']]

In [30]:
for_csv['forecast']  = pred
for_csv['verdict'] = np.where(pred >= 0.53, 1, 0)

In [31]:
for_csv= for_csv.reset_index().drop('index',axis=1)
for_csv

,card_num,application,target,forecast,verdict
0,2,379411,1,0.986334,1
1,2,379412,0,0.009127,0
2,2,379414,1,0.619936,1
3,2,379417,0,0.593263,1
4,2,379424,0,0.009127,0
...,...,...,...,...,...
1922953,403429824,379929,0,0.009127,0
1922954,403429824,379931,0,0.009127,0
1922955,403429824,379933,0,0.009127,0
1922956,403429824,379937,1,0.903094,1


In [ ]:
from sklearn.cluster import DBSCAN
claster = DBSCAN(eps=0.5, min_samples=10, 
                 metric='euclidean', metric_params=None, algorithm='auto', 
                 leaf_size=30, p=None, n_jobs=None).fit(X_train)

In [ ]:
for_csv.to_csv('output_1.csv')